In [1]:
!pip install -q transformers sentencepiece sentence-transformers ipywidgets nbformat

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.9 MB/s eta 0:00:00


In [10]:
!pip install -q transformers sentencepiece ipywidgets nbformat googletrans==4.0.0-rc1

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 5.49.1 requires httpx<1.0,>=0.24.1, but you have httpx 0.13.3 which is incompatible.
mcp 1.19.0 requires httpx>=0.27.1, but you have httpx 0.13.3 which is incompatible.
openai 1.109.1 requires httpx<1,>=0.23.0, but you have httpx 

In [11]:
# Rode no Colab
!pip install -q transformers sentencepiece sentence-transformers ipywidgets nbformat


In [13]:
# no Colab, apenas uma vez
!pip install -q wikipedia


  Preparing metadata (setup.py) ... done


In [4]:
# Arquivo gerado: colab_nlp_demo.py
# Este código deve ser executado em Google Colab (ou Jupyter). Ele cria uma UI simples com ipywidgets
# em português para escolher exemplos anexos, digitar texto, e aplicar tarefas NLP (geração, resumo, tradução, Q&A).
# Observação: o código instala dependências ao rodar no Colab (comentários indicam onde executar pip).

colab_code = r'''
# === Colab NLP Demo (Português) ===
# Instruções rápidas:
# 1) Copie e cole todo este bloco em uma célula no Colab (ou salve como .py e execute).
# 2) Execute a célula; a primeira vez pode demorar (instala pacotes e baixa modelos).
# 3) Use a interface para escolher um exemplo e testar.
#
# Nota sobre tradução: o fluxo é em português. Se um modelo só aceitar inglês, o texto será traduzido
# automaticamente para o inglês, passado ao modelo, e o resultado será traduzido de volta para o português.
#
# Recomendação de execução em Colab: execute cada bloco em ordem. Se quiser trocar modelos, altere as strings de modelo abaixo.

# --- (1) Instalar dependências (executar no Colab) ---
# !pip install -q transformers sentencepiece sentence-transformers ipywidgets nbformat
# Obs: se quiser tradução via googletrans em vez de MarianMT, descomente e instale:
# !pip install -q googletrans==4.0.0-rc1

# --- (2) Código principal ---
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
import os
import nbformat
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer, util

# --- Configurações de modelo (ajuste se desejar) ---
# Modelos T5/Marian recomendados para tradução e seq2seq. Eles serão baixados quando executados no Colab.
MODEL_SUMMARY = "facebook/bart-large-cnn"        # resumir (funciona bem em inglês)
MODEL_TRANSLATE_EN_PT = "Helsinki-NLP/opus-mt-en-ROMANCE"  # traduz EN -> ROMANCE (inclui pt)
MODEL_TRANSLATE_PT_EN = "Helsinki-NLP/opus-mt-ROMANCE-en"  # traduz ROMANCE -> EN (usar para pt->en)
MODEL_QA = "deepset/roberta-base-squad2"        # Q&A (inglês)
MODEL_GEN = "pierreguillou/t5-small-portuguese" # geração em português (seq2seq T5 fine-tuned em PT)
EMBEDDING_MODEL = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2" # embeddings multilingue

# --- Funções utilitárias ---
def load_notebook_text(path):
    # Extrai texto de células markdown e code de um .ipynb (se existir)
    try:
        nb = nbformat.read(path, as_version=4)
        texts = []
        for cell in nb.cells:
            if cell.cell_type in ("markdown", "code"):
                texts.append(cell.source)
        return "\n\n".join(texts)[:20000]  # limitar tamanho
    except Exception as e:
        return f"(erro ao ler {path}: {e})"

# Carregar lista de exemplos no diretório /mnt/data (Colab upload padrão)
EXAMPLES_DIR = "/mnt/data"
example_files = []
if os.path.exists(EXAMPLES_DIR):
    for f in os.listdir(EXAMPLES_DIR):
        if f.lower().endswith(".ipynb") or f.lower().endswith(".txt") or f.lower().endswith(".md"):
            example_files.append(f)
example_files = sorted(example_files)

# --- Carregadores preguiçosos para modelos (só carregam quando necessário) ---
_loaded = {}
def get_translator_pt_en():
    if "pt_en" not in _loaded:
        _loaded["pt_en_tokenizer"] = AutoTokenizer.from_pretrained(MODEL_TRANSLATE_PT_EN)
        _loaded["pt_en_model"] = AutoModelForSeq2SeqLM.from_pretrained(MODEL_TRANSLATE_PT_EN)
    return _loaded["pt_en_tokenizer"], _loaded["pt_en_model"]

def get_translator_en_pt():
    if "en_pt" not in _loaded:
        _loaded["en_pt_tokenizer"] = AutoTokenizer.from_pretrained(MODEL_TRANSLATE_EN_PT)
        _loaded["en_pt_model"] = AutoModelForSeq2SeqLM.from_pretrained(MODEL_TRANSLATE_EN_PT)
    return _loaded["en_pt_tokenizer"], _loaded["en_pt_model"]

def translate_pt_to_en(text):
    tok, model = get_translator_pt_en()
    inputs = tok(text, return_tensors="pt", truncation=True, max_length=512)
    outs = model.generate(**inputs, max_length=512)
    return tok.decode(outs[0], skip_special_tokens=True)

def translate_en_to_pt(text):
    tok, model = get_translator_en_pt()
    inputs = tok(text, return_tensors="pt", truncation=True, max_length=512)
    outs = model.generate(**inputs, max_length=512)
    return tok.decode(outs[0], skip_special_tokens=True)

def ensure_english(text):
    # heurística simples: se contém muitos caracteres acentuados e palavras PT comuns, traduzir
    pt_indicators = [" que ", " não ", " para ", " por ", " com ", " é ", " está ", " será "]
    if any(w in text.lower() for w in pt_indicators):
        return translate_pt_to_en(text), True
    return text, False

# --- Interface com ipywidgets ---
ex_select = widgets.Dropdown(
    options=["(Nenhum)"] + example_files,
    description="Exemplo:",
    layout=widgets.Layout(width="80%")
)

txt_area = widgets.Textarea(
    value="",
    placeholder="Digite ou cole o texto aqui...",
    description="Entrada:",
    layout=widgets.Layout(width="98%", height="160px")
)

task_select = widgets.Dropdown(
    options=["gerar_texto", "resumir", "traduzir_pt_en", "traduzir_en_pt", "pergunta_resposta", "semantic_search"],
    value="gerar_texto",
    description="Tarefa:"
)

model_select = widgets.Text(
    value=MODEL_GEN,
    description="Modelo:",
    layout=widgets.Layout(width="80%"),
    placeholder="Nome do modelo HF (opcional)"
)

btn = widgets.Button(description="Executar", button_style="primary")

out = widgets.Output(layout={'border': '1px solid black'})

# Função principal ao clicar
def on_execute(b):
    with out:
        clear_output()
        # preparar texto de entrada
        chosen = ex_select.value
        if chosen and chosen != "(Nenhum)":
            path = os.path.join(EXAMPLES_DIR, chosen)
            base_text = load_notebook_text(path)
            print(f"Usando exemplo: {chosen}\n--- Trecho do exemplo ---\n{base_text[:2000]}\n--- Fim do trecho ---\n")
            # se o usuário digitou algo, concatena
            if txt_area.value.strip():
                base_text = base_text + "\n\n" + txt_area.value.strip()
        else:
            base_text = txt_area.value.strip()
        if not base_text:
            print("ERRO: nenhum texto de entrada fornecido.")
            return

        task = task_select.value
        chosen_model = model_select.value.strip() or None

        # Gerar / Resumir / Traduzir / Q&A
        if task == "gerar_texto":
            # Geração seq2seq: usa modelo T5-like se disponível; roda em PT por padrão.
            model_name = chosen_model or MODEL_GEN
            print(f"Gerando texto com {model_name} ... (pode demorar ao baixar modelo)")
            gen_pipe = pipeline("text2text-generation", model=model_name, tokenizer=model_name, device=0 if "COLAB_GPU" in os.environ else -1)
            prompt = f"gerar: {base_text}"
            res = gen_pipe(prompt, max_length=512, num_return_sequences=1)
            out_text = res[0]["generated_text"] if isinstance(res, list) else str(res)
            print("Resultado:\n", out_text)

        elif task == "resumir":
            model_name = chosen_model or MODEL_SUMMARY
            # traduz para inglês se necessário (bart-large-cnn é em inglês)
            txt_en, translated = ensure_english(base_text)
            if translated:
                print("(Texto traduzido para inglês antes do resumo.)")
            print(f"Resumindo com {model_name} ...")
            summarizer = pipeline("summarization", model=model_name, tokenizer=model_name, device=0 if "COLAB_GPU" in os.environ else -1)
            summary = summarizer(txt_en, max_length=200, min_length=30, do_sample=False)
            summary_text = summary[0]["summary_text"]
            if translated:
                summary_text = translate_en_to_pt(summary_text)
            print("Resumo (pt):\n", summary_text)

        elif task == "traduzir_pt_en":
            print("Traduzindo PT -> EN ...")
            print(translate_pt_to_en(base_text))

        elif task == "traduzir_en_pt":
            print("Traduzindo EN -> PT ...")
            print(translate_en_to_pt(base_text))

        elif task == "pergunta_resposta":
            # Q&A: se base_text é contexto e usuário pergunta algo no final. Pedimos que o usuário escreva pergunta no final
            parts = base_text.split("\n")
            question = parts[-1] if len(parts)>0 else ""
            context = "\n".join(parts[:-1]) if len(parts)>1 else base_text
            print("Contexto (trecho):", context[:1000])
            # garantir inglês para modelo padrão (deepset/roberta-base-squad2)
            txt_en_ctx, ctx_translated = ensure_english(context)
            q_en, q_translated = ensure_english(question)
            qa_model = chosen_model or MODEL_QA
            qa = pipeline("question-answering", model=qa_model, tokenizer=qa_model, device=0 if "COLAB_GPU" in os.environ else -1)
            ans = qa(question=q_en, context=txt_en_ctx)
            answer = ans.get("answer", "")
            if q_translated or ctx_translated:
                answer = translate_en_to_pt(answer)
            print("Resposta:\n", answer)
            print("\nDetalhes do pipeline:", ans)

        elif task == "semantic_search":
            # Busca semântica simples: indexa input (pode ser múltiplos documentos separados por '---doc---')
            print("Rodando busca semântica (paraphrase-multilingual-MiniLM-L12-v2)...")
            embedder = SentenceTransformer(EMBEDDING_MODEL)
            docs = [d.strip() for d in base_text.split("---doc---") if d.strip()]
            doc_emb = embedder.encode(docs, convert_to_tensor=True)
            query = widgets.Text(value="", description="Consulta:")
            btn2 = widgets.Button(description="Buscar")
            out2 = widgets.Output()
            def on_search(b):
                with out2:
                    clear_output()
                    q = query.value.strip()
                    if not q:
                        print("Digite a consulta.")
                        return
                    q_emb = embedder.encode(q, convert_to_tensor=True)
                    hits = util.semantic_search(q_emb, doc_emb, top_k=5)[0]
                    for h in hits:
                        idx = h["corpus_id"]
                        score = h["score"]
                        print(f"--- Documento {idx} (score={score:.4f}) ---")
                        print(docs[int(idx)][:1000])
                        print("\n")
            btn2.on_click(on_search)
            display(query, btn2, out2)
            print("Separe documentos com '---doc---' no texto de entrada. Use o campo acima para consultar.")

        else:
            print("Tarefa não suportada.")

# ligar evento
btn.on_click(on_execute)

# Mostrar interface
display(HTML('<h3>Demo NLP (Português) — Colab</h3>'))
display(ex_select)
display(txt_area)
display(task_select)
display(model_select)
display(btn)
display(out)
'''

# Salvar arquivo no /mnt/data para o usuário baixar/usar
path = "/content/colab_nlp_demo.py"
with open(path, "w", encoding="utf-8") as f:
    f.write(colab_code)

# Listar arquivos no /mnt/data e mostrar o começo do script
import os, textwrap
files = os.listdir("/content")
preview = ""
with open(path, "r", encoding="utf-8") as f:
    preview = "".join([next(f) for _ in range(60)])  # mostra primeiras 60 linhas

print("Arquivos em /content:")
for fi in files:
    print(" -", fi)
print("\nArquivo criado:", path)
print("\nPreview do início do arquivo (60 linhas):\n")
print(preview)
print("\nPara usar: abra o Colab, faça upload dos exemplos (se ainda não estiverem em /mnt/data), copie todo o conteúdo do arquivo 'colab_nlp_demo.py' para uma célula e execute. Siga as instruções no topo do arquivo para instalar dependências.")



Arquivos em /content:
 - .config
 - colab_nlp_demo.py
 - sample_data

Arquivo criado: /content/colab_nlp_demo.py

Preview do início do arquivo (60 linhas):


# === Colab NLP Demo (Português) ===
# Instruções rápidas:
# 1) Copie e cole todo este bloco em uma célula no Colab (ou salve como .py e execute).
# 2) Execute a célula; a primeira vez pode demorar (instala pacotes e baixa modelos).
# 3) Use a interface para escolher um exemplo e testar.
#
# Nota sobre tradução: o fluxo é em português. Se um modelo só aceitar inglês, o texto será traduzido
# automaticamente para o inglês, passado ao modelo, e o resultado será traduzido de volta para o português.
#
# Recomendação de execução em Colab: execute cada bloco em ordem. Se quiser trocar modelos, altere as strings de modelo abaixo.

# --- (1) Instalar dependências (executar no Colab) ---
# !pip install -q transformers sentencepiece sentence-transformers ipywidgets nbformat
# Obs: se quiser tradução via googletrans em vez de MarianMT, desco

In [23]:
# CÓDIGO COMPLETO PARA COLOCAR NUMA CÉLULA ÚNICA DO COLAB
# Instalar antes se necessário:
# !pip install -q transformers sentencepiece sentence-transformers ipywidgets nbformat wikipedia

from IPython.display import display, clear_output, HTML
import ipywidgets as widgets
import os, nbformat, torch, warnings, re, time
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

warnings.filterwarnings("ignore")

# Diretório com exemplos
EXAMPLES_DIR = "/content"
def list_examples():
    files = []
    if os.path.exists(EXAMPLES_DIR):
        for f in os.listdir(EXAMPLES_DIR):
            if f.lower().endswith((".ipynb", ".txt", ".md")):
                files.append(f)
    return sorted(files)

def load_notebook_text(path):
    try:
        nb = nbformat.read(path, as_version=4)
        texts = []
        for cell in nb.cells:
            if cell.cell_type in ("markdown", "code"):
                texts.append(cell.source)
        return "\n\n".join(texts)[:20000]
    except Exception as e:
        return f"(erro ao ler {path}: {e})"

# Modelos e configurações
MODEL_PT_T5 = "unicamp-dl/ptt5-base-portuguese-vocab"  # geração em PT preferida
MODEL_FLAN = "google/flan-t5-base"                     # fallback instruído
MODEL_NEO = "EleutherAI/gpt-neo-125M"                  # fallback causal
MODEL_MT5 = "google/mt5-small"                         # fallback multilingual (tradução/resumo)
MODEL_SUMMARY = "facebook/bart-large-cnn"              # summarizer EN
MARIA_PT_EN = "Helsinki-NLP/opus-mt-pt-en"
MARIA_EN_PT = "Helsinki-NLP/opus-mt-en-pt"
MODEL_QA = "deepset/roberta-base-squad2"

device = 0 if torch.cuda.is_available() else -1
_loaded = {}

# Tenta carregar tradutor Marian; se não der, retorna (None, None)
def try_load_marien(name):
    key = f"trans_{name}"
    if key in _loaded:
        return _loaded[key]
    try:
        tok = AutoTokenizer.from_pretrained(name)
        model = AutoModelForSeq2SeqLM.from_pretrained(name)
        _loaded[key] = (tok, model)
        return tok, model
    except Exception:
        return None, None

# Tradução: tenta Marian, se não disponível usa MT5 como fallback generator (prompt)
def translate_pt_to_en(text):
    tok, model = try_load_marien(MARIA_PT_EN)
    if tok is not None:
        inputs = tok(text, return_tensors="pt", truncation=True, max_length=512)
        outs = model.generate(**inputs, max_length=512)
        return tok.decode(outs[0], skip_special_tokens=True)
    # fallback via MT5
    try:
        mtpipe = pipeline("text2text-generation", model=MODEL_MT5, tokenizer=MODEL_MT5, device=device)
        out = mtpipe(f"Translate to English: {text}", max_new_tokens=256, do_sample=False, num_return_sequences=1)
        return (out[0].get("generated_text") or out[0].get("text") or str(out[0])).strip()
    except Exception:
        return text

def translate_en_to_pt(text):
    tok, model = try_load_marien(MARIA_EN_PT)
    if tok is not None:
        inputs = tok(text, return_tensors="pt", truncation=True, max_length=512)
        outs = model.generate(**inputs, max_length=512)
        return tok.decode(outs[0], skip_special_tokens=True)
    # fallback via MT5
    try:
        mtpipe = pipeline("text2text-generation", model=MODEL_MT5, tokenizer=MODEL_MT5, device=device)
        out = mtpipe(f"Translate to Portuguese: {text}", max_new_tokens=256, do_sample=False, num_return_sequences=1)
        return (out[0].get("generated_text") or out[0].get("text") or str(out[0])).strip()
    except Exception:
        return text

def ensure_english_if_possible(text):
    pt_indicators = [" que ", " não ", " para ", " por ", " com ", " é ", " está ", " será ", " também "]
    if any(w in text.lower() for w in pt_indicators):
        try:
            return translate_pt_to_en(text), True
        except Exception:
            return text, False
    return text, False

# Helpers de geração segura
def is_bad_output(text, prompt):
    t = (text or "").strip()
    if not t:
        return True
    if len(t) < 10:
        return True
    p_short = prompt.strip().lower()[:40]
    if t.lower().startswith(p_short):
        return True
    words = re.findall(r'\w+', t.lower())
    if len(words) > 6 and len(set(words[-6:])) == 1:
        return True
    return False

def safe_generate_text(pipe, prompt, max_new_tokens=150, deterministic=True):
    gen_kwargs = {"max_new_tokens": max_new_tokens, "early_stopping": True, "no_repeat_ngram_size": 3, "repetition_penalty": 1.2}
    if deterministic:
        gen_kwargs["do_sample"] = False
    else:
        gen_kwargs.update({"do_sample": True, "top_p": 0.92, "temperature": 0.8})
    res = pipe(prompt, **gen_kwargs, num_return_sequences=1)
    if isinstance(res, list) and len(res) > 0:
        return res[0].get("generated_text") or res[0].get("text") or res[0].get("summary_text") or str(res[0])
    return str(res)

# UI widgets
ex_select = widgets.Dropdown(options=["(Nenhum)"] + list_examples(), description="Exemplo:", layout=widgets.Layout(width="80%"))
txt_area = widgets.Textarea(value="", placeholder="Digite ou cole o texto aqui (tema: esportes)", description="Entrada:", layout=widgets.Layout(width="98%", height="160px"))
task_select = widgets.Dropdown(options=[("Gerar texto","gerar_texto"),("Resumir","resumir"),("Traduzir PT->EN","traduzir_pt_en"),("Traduzir EN->PT","traduzir_en_pt"),("Pergunta/Resposta","pergunta_resposta")], value="gerar_texto", description="Tarefa:")
model_input = widgets.Text(value="", description="Modelo (opcional):", layout=widgets.Layout(width="80%"))
run_btn = widgets.Button(description="Executar", button_style="success")
out = widgets.Output(layout={'border': '1px solid black'})

# Função principal
def on_run(b):
    with out:
        clear_output()
        chosen = ex_select.value
        if chosen and chosen != "(Nenhum)":
            path = os.path.join(EXAMPLES_DIR, chosen)
            base_text = load_notebook_text(path)
            if txt_area.value.strip():
                base_text = base_text + "\n\n" + txt_area.value.strip()
        else:
            base_text = txt_area.value.strip()
        if not base_text:
            print("ERRO: nenhum texto de entrada fornecido.")
            return

        task = task_select.value
        chosen_model = model_input.value.strip() or None

        try:
            # ---------- GERAÇÃO (nova versão com Wikipedia como fonte principal) ----------
            if task == "gerar_texto":
                import wikipedia
                wikipedia.set_lang("pt")
                tema = base_text.strip()
                if not tema:
                    print("ERRO: nenhum tema informado.")
                else:
                    print(f"🔎 Buscando resumo sobre '{tema}' na Wikipedia (PT)...")
                    try:
                        results = wikipedia.search(tema, results=3)
                        if results:
                            page = wikipedia.page(results[0])
                            summary = page.summary
                            # limitar para evitar textos muito longos
                            paragraphs = summary.split("\n")
                            resumo_final = "\n\n".join(paragraphs[:5]).strip()
                            print("\n🟢 Resultado (Wikipedia):\n", resumo_final)
                        else:
                            print("❌ Nenhum resultado encontrado na Wikipedia, tentando gerar texto com modelo...")

                            # fallback - geração automática (somente se Wikipedia não encontrou nada)
                            model_name = chosen_model or MODEL_FLAN
                            gen_pipe = pipeline("text2text-generation", model=model_name, tokenizer=model_name, device=device)
                            prompt = (
                                f"Escreva um texto informativo e coerente em português sobre o tema '{tema}'. "
                                "Use linguagem natural e ao menos 5 frases completas."
                            )
                            res = gen_pipe(prompt, max_new_tokens=220, do_sample=True, top_p=0.92, temperature=0.9)
                            texto = res[0].get("generated_text") or res[0].get("text") or str(res[0])
                            print("\n🟢 Resultado (gerado):\n", texto.strip())
                    except Exception as e:
                        print("Erro ao buscar na Wikipedia:", e)

            # ---------- RESUMO ----------
            elif task == "resumir":
                model_name = chosen_model or MODEL_SUMMARY
                txt_en, translated = ensure_english_if_possible(base_text)
                if translated:
                    print("(texto detectado em português — traduzindo para inglês antes do resumo)")
                try:
                    summarizer = pipeline("summarization", model=model_name, tokenizer=model_name, device=device)
                    summary = summarizer(txt_en, max_length=200, min_length=30, do_sample=False)
                    summary_text = summary[0].get("summary_text", "")
                    if translated and summary_text:
                        try:
                            summary_text = translate_en_to_pt(summary_text)
                        except Exception:
                            pass
                    print("\n🟢 Resumo:\n", summary_text)
                except Exception as e:
                    print("(Aviso) summarizer falhou:", e)
                    # fallback via MT5
                    try:
                        mtpipe = pipeline("text2text-generation", model=MODEL_MT5, tokenizer=MODEL_MT5, device=device)
                        prompt = f"Resuma em português de forma clara: {base_text}"
                        out_text = safe_generate_text(mtpipe, prompt, max_new_tokens=180, deterministic=True)
                        out_text = out_text.replace("<extra_id_0>", "").strip()
                        print("\n🟢 Resumo (fallback):\n", out_text)
                    except Exception as e2:
                        print("Fallback de resumo falhou:", e2)

            # ---------- TRADUÇÃO PT->EN ----------
            elif task == "traduzir_pt_en":
                try:
                    res = translate_pt_to_en(base_text)
                    print("\n🟢 Tradução PT->EN:\n", res)
                except Exception as e:
                    print("❌ Erro na tradução PT->EN:", e)
                    # fallback mt5
                    try:
                        mtpipe = pipeline("text2text-generation", model=MODEL_MT5, tokenizer=MODEL_MT5, device=device)
                        out_text = safe_generate_text(mtpipe, f"Translate to English: {base_text}", max_new_tokens=200, deterministic=True)
                        print("\n🟢 Tradução (fallback mt5):\n", out_text)
                    except Exception as e2:
                        print("Fallback de tradução falhou:", e2)

            # ---------- TRADUÇÃO EN->PT ----------
            elif task == "traduzir_en_pt":
                try:
                    res = translate_en_to_pt(base_text)
                    print("\n🟢 Tradução EN->PT:\n", res)
                except Exception as e:
                    print("❌ Erro na tradução EN->PT:", e)
                    try:
                        mtpipe = pipeline("text2text-generation", model=MODEL_MT5, tokenizer=MODEL_MT5, device=device)
                        out_text = safe_generate_text(mtpipe, f"Translate to Portuguese: {base_text}", max_new_tokens=200, deterministic=True)
                        print("\n🟢 Tradução (fallback mt5):\n", out_text)
                    except Exception as e2:
                        print("Fallback de tradução falhou:", e2)

            # ---------- PERGUNTA/RESPOSTA ----------
            elif task == "pergunta_resposta":
                parts = base_text.strip().split("\n")
                if len(parts) > 1:
                    question = parts[-1].strip()
                    context = "\n".join(parts[:-1]).strip()
                else:
                    question = parts[0].strip()
                    context = ""

                if context:
                    print("Respondendo com base no contexto (extractive QA)...")
                    try:
                        ctx_en, ctx_translated = ensure_english_if_possible(context)
                    except Exception:
                        ctx_en, ctx_translated = context, False
                    try:
                        q_en, q_translated = ensure_english_if_possible(question)
                    except Exception:
                        q_en, q_translated = question, False
                    qa = pipeline("question-answering", model=chosen_model or MODEL_QA, tokenizer=chosen_model or MODEL_QA, device=device)
                    ans = qa(question=q_en, context=ctx_en)
                    answer = ans.get("answer","")
                    if (ctx_translated or q_translated) and answer:
                        try:
                            answer = translate_en_to_pt(answer)
                        except Exception:
                            pass
                    print("\n🟢 Resposta:", answer)
                    print("\nDetalhes:", ans)
                else:
                    print("Sem contexto — tentando resposta pela Wikipedia (consulta rápida)...")
                    try:
                        import wikipedia
                        wikipedia.set_lang("pt")
                        hits = wikipedia.search(question, results=5)
                        if not hits:
                            raise Exception("Nenhum resultado na Wikipedia")
                        page_title = hits[0]
                        summary = wikipedia.summary(page_title, sentences=3)
                        print("\n🟢 Resposta (via Wikipedia - verifique):\n", summary)
                    except Exception as e:
                        print("(Wikipedia falhou ou sem resultado):", e)
                        print("Fallback: gerando resposta com modelo (pode não ser 100% confiável).")
                        try:
                            gen = pipeline("text2text-generation", model=MODEL_FLAN, tokenizer=MODEL_FLAN, device=device)
                            prompt = f"Instruction: Responda de forma objetiva e curta.\nInput: {question}\nOutput:"
                            ans = safe_generate_text(gen, prompt, max_new_tokens=120, deterministic=False)
                            ans = ans.replace("<extra_id_0>", "").strip()
                            print("\n🟢 Resposta (gerada):\n", ans)
                        except Exception as e2:
                            print("Fallback gerador falhou:", e2)

            else:
                print("Tarefa não suportada.")
        except Exception as e:
            print("Erro durante a execução:", e)

# ligar evento
run_btn.on_click(on_run)

# Mostrar UI
display(HTML("<h3>Demo NLP — Versão Final (Colab) — Domínio: Esportes</h3><p>Foco: Geração em português, resumo, tradução e QA. Recomendo ativar GPU (Runtime → Change runtime type → GPU).</p>"))
display(ex_select)
display(txt_area)
display(task_select)
display(model_input)
display(run_btn)
display(out)


Dropdown(description='Exemplo:', layout=Layout(width='80%'), options=('(Nenhum)', 'raw_debug.txt'), value='(Ne…

Textarea(value='', description='Entrada:', layout=Layout(height='160px', width='98%'), placeholder='Digite ou …

Dropdown(description='Tarefa:', options=(('Gerar texto', 'gerar_texto'), ('Resumir', 'resumir'), ('Traduzir PT…

Text(value='', description='Modelo (opcional):', layout=Layout(width='80%'))

Button(button_style='success', description='Executar', style=ButtonStyle())

Output(layout=Layout(border='1px solid black'))